# 유튜브 댓글 크롤링 (2)
### keyword: 라이프해커자청

In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
from tqdm import tqdm

In [12]:
keyword = "라이프해커자청"

In [13]:
v_url_list = []
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
url = 'https://www.youtube.com/results?search_query=' + quote(keyword)
driver.get(url)
time.sleep(2)

# channel_url, channel_id, channel_name 가져오기
main_link = driver.find_element_by_id("main-link")
channel_url = main_link.get_attribute("href")
c_idx = channel_url.rfind("/") + 1
channel_id = channel_url[c_idx:]
channel_name = main_link.find_element_by_id("text-container").text

# 채널 내 '동영상' 으로 이동
main_link.click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()
time.sleep(1)

# 스크롤 끝까지 내리기
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
driver.execute_script("window.scrollTo(0, 350);")
while True: 
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
    time.sleep(3) 
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
    if new_page_height == last_page_height: 
        break 
    last_page_height = new_page_height

lis = driver.find_elements_by_id('thumbnail')

for li in lis:
    video_url = li.get_attribute("href")
    v_url_list.append(video_url)

driver.close()

In [14]:
print(len(v_url_list), "\n", v_url_list)

50 
 ['https://www.youtube.com/watch?v=AUCKb5b5UNs', 'https://www.youtube.com/watch?v=1Jgi2zSavT0', 'https://www.youtube.com/watch?v=4Yfu0-XHGgU', 'https://www.youtube.com/watch?v=SR2SWfIXt7s', 'https://www.youtube.com/watch?v=7qBpW5hHLho&list=PLdaXIYLRupw5LlrPXnDeBzbI8N8j6tnei', 'https://www.youtube.com/watch?v=KNqywKwNUSA', 'https://www.youtube.com/watch?v=WBlWGf70H1g', 'https://www.youtube.com/watch?v=07E2E0fiT54', 'https://www.youtube.com/watch?v=cnDoYsVJhfA', 'https://www.youtube.com/watch?v=0IYxVpuL-Qw', 'https://www.youtube.com/watch?v=GPuSUBHUDFA', 'https://www.youtube.com/watch?v=F0ZHHYl6Cyg', 'https://www.youtube.com/watch?v=fgdSQqDa7sE', 'https://www.youtube.com/watch?v=MLwDArWupOw', 'https://www.youtube.com/watch?v=f2hYXC1A5Nc', 'https://www.youtube.com/watch?v=1dsWprRplgU', 'https://www.youtube.com/watch?v=qPgvZrJwVHs', 'https://www.youtube.com/watch?v=GgP_TTGisBY', 'https://www.youtube.com/watch?v=imk6pmLy2zE', 'https://www.youtube.com/watch?v=PcFdkiYnXTI', 'https://www.y

In [15]:
comment_list = []; 
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
for url in tqdm(v_url_list):
    if url == None:
        continue
    else:
        driver.get(url)
        time.sleep(1)

        # 스크롤 끝까지 내리기
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(0, 350);")
        time.sleep(2)
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 댓글 수집
        contents = driver.find_element_by_class_name("style-scope ytd-item-section-renderer")
        try:
            comments = contents.find_elements_by_tag_name('ytd-expander')
            for comment in comments:
                text = comment.find_element_by_tag_name('div').text
                comment_list.append(text)
        except NoSuchElementException:
            continue

driver.close()

100%|██████████| 50/50 [51:20<00:00, 61.60s/it]


In [16]:
len(comment_list)

15599

### 댓글 원본 저장(전처리 X)

In [17]:
import pandas as pd 
df = pd.DataFrame({
    'comments': comment_list
})
df.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요. 아무튼 사업에 대한 스킬적인 것 보...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화=홍익인간 (널리 인간을 이롭게하라)
3,많은 인생의 실패자들은 포기할 때 자신이 성공에서 얼마나 가까이 있었는지 모른다.\...
4,진짜 성공한분들은 거의 비슷한 말을 하지만 들어야하는 사람들은 준비가 되어있나 싶다...


In [18]:
df.to_csv('comments_origin2.txt', sep='\t', index=False)

### 데이터 전처리

In [3]:
import pandas as pd 
df_test = pd.read_table('./comments_origin2.txt')
df_test.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요. 아무튼 사업에 대한 스킬적인 것 보...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화=홍익인간 (널리 인간을 이롭게하라)
3,많은 인생의 실패자들은 포기할 때 자신이 성공에서 얼마나 가까이 있었는지 모른다.\...
4,진짜 성공한분들은 거의 비슷한 말을 하지만 들어야하는 사람들은 준비가 되어있나 싶다...


In [4]:
df_test.shape

(15599, 1)

In [5]:
df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_test.head()

<ipython-input-5-4884171cc048>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요 아무튼 사업에 대한 스킬적인 것 보다...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화홍익인간 널리 인간을 이롭게하라
3,많은 인생의 실패자들은 포기할 때 자신이 성공에서 얼마나 가까이 있었는지 모른다 토...
4,진짜 성공한분들은 거의 비슷한 말을 하지만 들어야하는 사람들은 준비가 되어있나 싶다...


In [6]:
# 'comments' 값 공백 여부 확인
drop_idx = df_test[df_test.comments == ""].index
print(len(drop_idx), drop_idx)

102 Int64Index([  929,  1002,  1040,  1049,  1319,  1339,  1670,  2180,  2264,
             2269,
            ...
            14303, 14308, 14328, 14349, 14517, 14527, 14543, 15268, 15318,
            15588],
           dtype='int64', length=102)


In [7]:
# 공백 값을 가지고 있는 해당 인덱스 행 제거
df_test.drop(drop_idx, inplace=True)
df_test.shape

(15497, 1)

In [8]:
# 중복 제거
df_test.drop_duplicates(['comments'], inplace=True)
df_test.shape

(11761, 1)

In [9]:
# null값 여부 확인
df_test.isnull().sum()

comments    0
dtype: int64

In [10]:
df_test.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요 아무튼 사업에 대한 스킬적인 것 보다...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화홍익인간 널리 인간을 이롭게하라
3,많은 인생의 실패자들은 포기할 때 자신이 성공에서 얼마나 가까이 있었는지 모른다 토...
4,진짜 성공한분들은 거의 비슷한 말을 하지만 들어야하는 사람들은 준비가 되어있나 싶다...


In [11]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 11761


In [12]:
df_test.to_csv('comments_result2.txt', sep='\t', index=False)